<a href="https://colab.research.google.com/github/mahb97/Howl-Ginsberg-poemsssss/blob/main/Visionarytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install torch==2.4.1 bitsandbytes==0.43.3 transformers==4.44.2 datasets==2.21.0 peft==0.12.0 trl==0.9.6 accelerate==0.34.2

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, SFTConfig
import torch, os

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # or "TinyLlama/TinyLlama-1.1B-intermediate-step-955k-2T"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token

data = load_dataset("json", data_files={"/content/ginsberg.jsonl": "train"}, split="train")

def format_example(ex):
    return {"text": ex["text"].strip()}

data = data.map(format_example, remove_columns=data.column_names)

# Be crazy dumbsaint of the mind
bnb_cfg = dict(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", **bnb_cfg)

# call my daughter LoRA
lora = LoraConfig(
    r=16, lora_alpha=32, target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
)

# Scribbled secret notebooks, and wild typewritten pages, for yr own joy
cfg = SFTConfig(
    output_dir="/content/tinyllama-ginsberg-lora",
    num_train_epochs=2,               # low start; poetry overfits quickly
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    bf16=True,
    logging_steps=20,
    save_steps=200,
    save_total_limit=2,
    packing=True,
    max_seq_length=1024,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    tokenizer=tokenizer,
    peft_config=lora,
    formatting_func=lambda batch: [x["text"] for x in batch],
    args=cfg,
)

trainer.train()
trainer.save_model()  # saves LoRA adapters

from peft import PeftModel
base = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="cpu")
merged = PeftModel.from_pretrained(base, "/content/tinyllama-ginsberg-lora").merge_and_unload()
merged.save_pretrained("/content/tinyllama-ginsberg-merged", safe_serialization=True)
tokenizer.save_pretrained("/content/tinyllama-ginsberg-merged")


In [ ]:
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import torch

base = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_4bit=True)
tok  = AutoTokenizer.from_pretrained(model_name)
ft   = PeftModel.from_pretrained(base, "/content/tinyllama-ginsberg-lora")

prompt = "Write a poem about cobblestones and rain.\n\n"
ids = tok(prompt, return_tensors="pt").to(ft.device)
streamer = TextStreamer(tok)
out = ft.generate(
    **ids,
    max_new_tokens=200,
    do_sample=True,
    temperature=0.9,
    top_p=0.95,
    repetition_penalty=1.1,
    eos_token_id=tok.eos_token_id
)
print(tok.decode(out[0], skip_special_tokens=True))
